In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from scipy.stats import skew
from scipy.stats import kurtosis
import statistics

import plotly.graph_objects as go

<ipython-input-1-27447f67d8b1>:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-1-27447f67d8b1>:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
app = dash.Dash(__name__)

# importing dataset

dataset = pd.read_csv('feeds_SACP1_old_2021-09-09.csv')
dataset_final = dataset[['Ambient Temp (degC)','Humidity (%)', 'Soil Moisture (%)', 'Capacitor Voltage (mV)', 'Soil Temperature (degC)', 'SACP Cell Current (mA)', 'Soil Voltage (V)', 'Cell Power (mW)']]

# seperating dataset based on time day month and year
dataset['created_at'] = pd.to_datetime(dataset['created_at'])
dataset['year'] = dataset['created_at'].dt.year
dataset['month'] = dataset['created_at'].dt.month
dataset['day'] = dataset['created_at'].dt.day

dataset = dataset.drop(dataset[dataset['SACP Cell Current (mA)'] > 100].index)
dataset = dataset.drop(dataset[dataset['Soil Voltage (V)'] > 3].index)
dataset = dataset.drop(dataset[dataset['Cell Power (mW)'] > 100].index)
dataset

C:\Users\DEEP\anaconda3\envs\tf_gpu\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\DEEP\anaconda3\envs\tf_gpu\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,created_at,entry_id,Ambient Temp (degC),Humidity (%),Soil Moisture (%),Capacitor Voltage (mV),Soil Temperature (degC),SACP Cell Current (mA),Soil Voltage (V),Cell Power (mW),year,month,day
0,2020-06-18 23:29:49,1,20.6875,47.49635,99,2907.74194,22.5000,25.9,0.2946,8,2020,6,18
1,2020-06-18 23:30:58,2,20.6250,47.52944,99,2909.03223,22.5000,26.0,0.2946,8,2020,6,18
2,2020-06-18 23:32:06,3,20.6250,47.55913,99,2909.03223,22.5000,26.0,0.2946,8,2020,6,18
3,2020-06-18 23:33:15,4,20.5625,47.59531,99,2912.90332,22.5000,26.0,0.2946,8,2020,6,18
4,2020-06-18 23:34:23,5,20.4375,47.60892,99,2911.61279,22.4375,26.0,0.2946,8,2020,6,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380356,2021-07-26 19:11:20,380357,26.1250,88.10663,100,2925.48389,24.9375,16.4,1.6400,0,2021,7,26
380357,2021-07-26 19:12:30,380358,26.0625,88.35374,100,2874.19360,24.9375,16.4,1.6400,0,2021,7,26
380358,2021-07-26 19:13:40,380359,26.0625,88.63950,100,2889.67749,24.9375,16.4,1.6400,0,2021,7,26
380359,2021-07-26 19:14:49,380360,25.9375,88.89094,100,3067.41943,24.8750,16.3,1.6300,0,2021,7,26


In [3]:
columns = ['Ambient Temp (degC)', 'Humidity (%)', 'Soil Moisture (%)', 'Capacitor Voltage (mV)', 'Soil Temperature (degC)', 'SACP Cell Current (mA)', 'Soil Voltage', 'Cell Power (mW)']
cols2 = ['created_at']
available_indicators = columns
available_indicators_2 = cols2

In [4]:
app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators_2],
                value='Capacitor Voltage (mV)'
            )], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
            dcc.Dropdown(
                id='yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='created_at'
            )], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
        dcc.Graph(id = 'indicator-graphic')
])

In [ ]:
@app.callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name):

    fig = px.scatter(x=dataset[xaxis_column_name],
                     y=dataset[yaxis_column_name])

    return fig


if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
